In [ ]:
import pandas as pd
import numpy as np

DATA DESCRIPTION
```
file name -> Columns
quater-i.csv -> ['order_id', 'quantity', 'item_id', 'choice_description_id' 'item_price']
items.csv -> ['item_id', 'item_name']
```
Dataset Link - https://drive.google.com/drive/folders/1Z0kaFybvgFeczeUj4dldUnhTdloLqLsL?usp=share_link

In [ ]:
q1= pd.read_csv('quarter-1.csv')
q2 = pd.read_csv('quarter-2.csv')
q3 = pd.read_csv('quarter-3.csv')
items = pd.read_csv('items.csv')

###`Q:1-5`
1. You are given three quater files, your job is to append these three files and make a single dataframe.
2. Have a index as Q-1 Q-2 Q-3 for respective quater files in the dataframe
3. Your are given a file items.csv which has item_id and item_name. Find out most sold items in each quarter.
4. Find out items which has made most revenue in each quarter.
5. Find out avg order price of each quarter.

***Note: item_price is given as str with $ sign, in earlier task you have converted this to rupees, here too first convert item_price field in rupees.***

In [ ]:
# Add index labels for the quarters
q1['Quarter'] = 'Q-1'
q2['Quarter'] = 'Q-2'
q3['Quarter'] = 'Q-3'

data = pd.concat([q1, q2, q3], ignore_index=True)
data = data.merge(items, on='item_id', how='left')

# Convert 'item_price' to rupees
data['item_price'] = data['item_price'].str.replace('[\$,]', '', regex=True).astype(float) * 80

# Calculate total revenue for each row
data['total_revenue'] = data['quantity'] * data['item_price']

# 1. Most sold items in each quarter
most_sold_items = data.groupby(['Quarter', 'item_id', 'item_name'])['quantity'].sum().reset_index()
most_sold_items = most_sold_items.sort_values(by=['Quarter', 'quantity'], ascending=[True, False]).groupby('Quarter').head(1)

# 2. Items with the most revenue in each quarter
most_revenue_items = data.groupby(['Quarter', 'item_id', 'item_name'])['total_revenue'].sum().reset_index()
most_revenue_items = most_revenue_items.sort_values(by=['Quarter', 'total_revenue'], ascending=[True, False]).groupby('Quarter').head(1)

# 3. Average order price in each quarter
avg_order_price = data.groupby('Quarter').apply(lambda x: (x['quantity'] * x['item_price']).sum() / x['quantity'].sum()).reset_index(name='avg_order_price')

In [ ]:
most_sold_items

,Quarter,item_id,item_name,quantity
4,Q-1,4,Chicken Bowl,367
53,Q-2,4,Chicken Bowl,394


In [ ]:
most_revenue_items

,Quarter,item_id,item_name,total_revenue
4,Q-1,4,Chicken Bowl,308190.4
53,Q-2,4,Chicken Bowl,335380.0


In [ ]:
avg_order_price

,Quarter,avg_order_price
0,Q-1,652.114706
1,Q-2,611.166719


###`Q-6` From the IPL wala dataset you have to find the Purple cap holder each season.

*Note: Bowler with most no. wickets in a season gets purple cap. If more than one bowler have same no of wickets in the season, one with least ecomnomy among them is purple cap holder.*

Bowler's Economy = runs-conceded per six balls

In [ ]:

ball_by_ball = pd.read_csv('IPL_Ball_by_Ball_2008_2022.csv')
match_data = pd.read_csv('IPL_Matches_2008_2022.csv')

In [ ]:
ball_by_ball.head(1)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [ ]:
match_data.head(1)

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon


In [ ]:

ipl = ball_by_ball.merge(match_data[['ID', 'Season']], on='ID', how='left')
ipl.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,Season
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,2022
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,2022


In [ ]:
purple_cap_holders = (ipl[ipl['isWicketDelivery'] == 1]
    .groupby(['Season', 'bowler'])
    .agg(wickets=('player_out', 'count'), runs_conceded=('total_run', 'sum'), valid_balls=('ballnumber', 'count'))
    .reset_index()
    .merge(
        ipl[ipl['extra_type'].isnull()].groupby(['Season', 'bowler'])
        .agg(valid_balls=('ballnumber', 'count'), runs_conceded=('total_run', 'sum'))
        .reset_index(),
        on=['Season', 'bowler'], how='left')
    .assign(overs=lambda df: df['valid_balls_x'] / 6, economy=lambda df: df['runs_conceded_y'] / (df['valid_balls_x'] / 6)) # Use valid_balls_x instead of valid_balls
    .sort_values(by=['Season', 'wickets', 'economy'], ascending=[True, False, True])
    .drop_duplicates(subset=['Season'], keep='first')
    [['Season', 'bowler', 'wickets', 'economy']]
).sort_values(by='wickets', ascending=False)

purple_cap_holders

,Season,bowler,wickets,economy
1232,2021,HV Patel,35,72.685714
519,2013,DJ Bravo,34,84.705882
1161,2020/21,K Rabada,32,99.750000
375,2011,SL Malinga,30,69.600000
447,2012,M Morkel,30,86.800000
1077,2019,K Rabada,29,75.103448
1402,2022,YS Chahal,29,104.689655
708,2015,DJ Bravo,28,87.857143
879,2017,B Kumar,28,76.500000
962,2018,AJ Tye,28,92.142857


###`Q-7:` Best bowler in death overs.
*Note: Have taken most no. of wickets in case of tie with least economy*

Death Overs - [16-20]

In [ ]:
ipl.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'Season'],
      dtype='object')

In [ ]:
best_death_bowler = (ipl[(ipl['overs'] >= 16) & (ipl['overs'] <= 20)]
    .groupby('bowler')
    .agg(wickets=('player_out', 'count'), runs_conceded=('total_run', 'sum'), valid_balls=('ballnumber', 'count'))
    .reset_index()
    .merge(ipl[(ipl['overs'] >= 16) & (ipl['overs'] <= 20) & (ipl['extra_type'].isnull())]
        .groupby('bowler').agg(valid_balls=('ballnumber', 'count'), runs_conceded=('total_run', 'sum')).reset_index(),
        on='bowler', how='left')
    .assign(overs=lambda df: df['valid_balls_y'] / 6,
            economy=lambda df: df['runs_conceded_y'] / (df['valid_balls_y'] / 6))
    .sort_values(by=['wickets', 'economy'], ascending=[False, True])
    .head(1)
    [['bowler', 'wickets', 'economy']]
)

best_death_bowler

,bowler,wickets,economy
85,DJ Bravo,115,9.480329


###`Q-8` Batsman record season wise

Make a function which takes a input `batsman_name` and it returns a dataframe.
Columns of the data frame are - `['Season','Innings', 'TotalRuns', 'Avg', 'HighestScore','StrikeRate']`.
* In result make `Season` column as index.

* Avg - total_runs/ no of time got out. - player_out column will help.
* StrikeRate -(total_runs/ balls faced) * 100- wides are not included in batsman ball faced counts. No balls are included. -> Extra_type column will help
* Batsman Can score runs on No Balls.
* Batsman can get out on No Ball or Wides. And even while being on non-striker. Keep these things in mind before masking.

In [ ]:
def batsman_record(batsman_name):
    return (ipl[ipl['batter'] == batsman_name]
        .groupby('Season')
        .agg(
            Innings=('ID', 'nunique'),
            TotalRuns=('batsman_run', 'sum'),
            Avg=('player_out', lambda x: x.count() if x.count() > 0 else None),  # If no outs, avg is None
            HighestScore=('batsman_run', 'max'),
            BallsFaced=('ballnumber', lambda x: (ipl[(ipl['batter'] == batsman_name) & (ipl['extra_type'] != 'wides')])['ballnumber'].count())
        )
        .assign(Avg=lambda df: df['TotalRuns'] / df['Avg'],
                StrikeRate=lambda df: (df['TotalRuns'] / df['BallsFaced']) * 100)
        # removed set_index as Season is already the index due to groupby
        [['Innings', 'TotalRuns', 'Avg', 'HighestScore', 'StrikeRate']]
    ).sort_values(by='Avg', ascending=False)


batsman_record('V Kohli')

,Innings,TotalRuns,Avg,HighestScore,StrikeRate
Season,,,,,
2016,16,973,81.083333,6,18.955776
2013,16,639,45.642857,6,12.448860
2018,14,530,44.166667,6,10.325346
2011,16,557,42.846154,6,10.851354
2020/21,15,471,42.818182,6,9.175921
2015,16,505,36.071429,6,9.838301
2019,14,464,33.142857,6,9.039548
2012,15,364,30.333333,6,7.091370
2021,15,405,28.928571,6,7.890123


from matplotlib import pyplot as plt
_df_0['Innings'].plot(kind='hist', bins=20, title='Innings')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['TotalRuns'].plot(kind='hist', bins=20, title='TotalRuns')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Avg'].plot(kind='hist', bins=20, title='Avg')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['StrikeRate'].plot(kind='hist', bins=20, title='StrikeRate')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['HighestScore']
  ys = series['StrikeRate']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_10.sort_values('HighestScore', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('HighestScore')
_ = plt.ylabel('StrikeRate')

from matplotlib import pyplot as plt
_df_11['Innings'].plot(kind='line', figsize=(8, 4), title='Innings')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['TotalRuns'].plot(kind='line', figsize=(8, 4), title='TotalRuns')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['Avg'].plot(kind='line', figsize=(8, 4), title='Avg')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['StrikeRate'].plot(kind='line', figsize=(8, 4), title='StrikeRate')
plt.gca().spines[['top', 'right']].set_visible(False)

###`Q-9` Using both dataset, make a dataframe as described below

Data Frame columns-> `['PlayerOfThematch', 'BattingFigure', 'BowlingFigure']`

* BattingFigure->`<runs>/<balls>`
* BowlingFigure->`<wicket>/<runs-conceded>`

DataFrame should have one record for each match.

Say 'V Kohli' got POM award then in dataset include his batting figure of that match. Say he scored 112runs in 76 balls. And he hasn't bowled so Bowling Figure will be NaN
```
PlayerOfThematch BattingFigure BowlingFigure
V Kohli          112/76         nan  

```


In [ ]:
import pandas as pd

def get_player_figures(match_data, ball_by_ball):
    merged_data = ball_by_ball.merge(match_data[['ID', 'Player_of_Match']], left_on='ID', right_on='ID', how='left')

    def compute_figures(player, match_id):
        match_data = merged_data[(merged_data['Player_of_Match'] == player) & (merged_data['ID'] == match_id)]

        # Batting figures: Total runs and balls faced (excluding wides)
        batting_data = match_data[match_data['batter'] == player]
        runs_scored = batting_data['batsman_run'].sum()
        balls_faced = batting_data[batting_data['extra_type'] != 'wides']['ballnumber'].count()
        batting_figure = f"{runs_scored}/{balls_faced}" if balls_faced > 0 else None

        # Bowling figures: Wickets and runs conceded
        bowling_data = match_data[match_data['bowler'] == player]
        wickets = bowling_data[bowling_data['isWicketDelivery'] == 1]['player_out'].count()
        runs_conceded = bowling_data['total_run'].sum()
        bowling_figure = f"{wickets}/{runs_conceded}" if bowling_data.shape[0] > 0 else None

        return pd.Series([batting_figure, bowling_figure])

    match_summary = match_data[['ID', 'Player_of_Match']].drop_duplicates().copy()
    match_summary[['BattingFigure', 'BowlingFigure']] = match_summary.apply(
        lambda row: compute_figures(row['Player_of_Match'], row['ID']), axis=1
    )

    return match_summary[['Player_of_Match', 'BattingFigure', 'BowlingFigure']].rename(columns={'Player_of_Match': 'PlayerOfTheMatch'})

get_player_figures(match_data, ball_by_ball)

,PlayerOfTheMatch,BattingFigure,BowlingFigure
0,HH Pandya,34/30,3/18
1,JC Buttler,106/60,None
2,RM Patidar,112/54,None
3,DA Miller,68/38,None
4,Harpreet Brar,None,3/26
...,...,...,...
945,DJ Hussey,38/43,1/35
946,MV Boucher,39/19,None
947,MF Maharoof,None,2/14
948,MEK Hussey,116/54,None


## **Questions Based on Iris Dataset**

- **Sepal All:** https://docs.google.com/spreadsheets/d/e/2PACX-1vT58ekmHTwptX7Bs4QOy6YByA1HMvYTACeeIjrKhHE0Pg1K_3egewHMKMh02zN9D5-yHVXfvuaa3s5u/pub?gid=2028782809&single=true&output=csv
    - **Unnamed: 0:** Unused column. This column is created when creating this sub-dataset.
    - **Id:** Id of the records.
    - **SepalLengthCm:** Sepal length of flowers in cm
    - **SepalWidthCm:** Sepal width of flowers in cm

- **Petal All:** https://docs.google.com/spreadsheets/d/e/2PACX-1vQinLXShrOz4ExNaW1bSQVuvbbhIzJW7G0kkkD2SvqSD6STjLrQQiftgI7BGe10sBZi0CNr2_sJpQAz/pub?gid=1580010789&single=true&output=csv
    - **Unnamed: 0:** Unused column. This column is created when creating this sub-dataset.
    - **Id:** Id of the records.
    - **PetalLengthCm:** Petal length of flowers in cm
    - **PetalWidthCm:** Petal width of flowers in cm

- **Iris Virginica:** https://docs.google.com/spreadsheets/d/e/2PACX-1vSK39MwduGPHYNgw5yViezoLYCVDKMCWIHzjnt3GZNaxHPFOQLr2q6no_tyqTsOk-VfXleslfGVe9eJ/pub?gid=314231613&single=true&output=csv
    - **Unnamed: 0:** Unused column. This column is created when creating the sub-dataset.
    - **Id:** Id of the records.
    - **Species:** Name of this species.

- **Iris Versicolor:** https://docs.google.com/spreadsheets/d/e/2PACX-1vTcSFgLnabqIrgIc5WlwvnbbvyyJsgZjR-0E0-4TR-5aHgv_0EP6yNWglkkls3AXM2qHCR5VYzWCoTM/pub?gid=715607857&single=true&output=csv
    - **Unnamed: 0:** Unused column. This column is created when creating the sub-dataset.
    - **Id:** Id of the records.
    - **Species:** Name of this species.

- **Iris Setosa:** https://docs.google.com/spreadsheets/d/e/2PACX-1vSjqJpdgy2X_oDUUqQ0sSaFKqnnf8MYU4KgJSYgHaHmq0Wb1weMOsJXh-rICHmkLcaTkOwzMYLeh959/pub?gid=2003684803&single=true&output=csv
    - **Unnamed 0:** Unused column. This column is created when creating the sub-dataset.
    - **Id:** Id of the records.
    - **Species:** Name of this species.

In [ ]:
import pandas as pd
sepal_all = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vT58ekmHTwptX7Bs4QOy6YByA1HMvYTACeeIjrKhHE0Pg1K_3egewHMKMh02zN9D5-yHVXfvuaa3s5u/pub?gid=2028782809&single=true&output=csv")
petal_all = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQinLXShrOz4ExNaW1bSQVuvbbhIzJW7G0kkkD2SvqSD6STjLrQQiftgI7BGe10sBZi0CNr2_sJpQAz/pub?gid=1580010789&single=true&output=csv")

virginica = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSK39MwduGPHYNgw5yViezoLYCVDKMCWIHzjnt3GZNaxHPFOQLr2q6no_tyqTsOk-VfXleslfGVe9eJ/pub?gid=314231613&single=true&output=csv")
versicolor = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTcSFgLnabqIrgIc5WlwvnbbvyyJsgZjR-0E0-4TR-5aHgv_0EP6yNWglkkls3AXM2qHCR5VYzWCoTM/pub?gid=715607857&single=true&output=csv")
setosa = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSjqJpdgy2X_oDUUqQ0sSaFKqnnf8MYU4KgJSYgHaHmq0Wb1weMOsJXh-rICHmkLcaTkOwzMYLeh959/pub?gid=2003684803&single=true&output=csv")


,Unnamed: 0,Id,Species
0,100,101,Iris-virginica
1,101,102,Iris-virginica
2,102,103,Iris-virginica
3,103,104,Iris-virginica
4,104,105,Iris-virginica


### `Q-9:` Plot a bar chart of the average Sepal Length  of Virginica and average Petal length of Setosa flower.

### `Q-10:` Create the complete dataset by uisng the below datasets:
- virginica
- versicolor
- setosa
- sepal all
- petal all

This dataset should have these below column names in order:
1. Id
2. Species
3. SepalLengthCm
4. SepalWidthCm
5. PetalLengthCm
6. PetalWidthCm

Also, the dataset should be shuffled means the `Id` column should not be in increasing or decreasing order. So, make a dataset which has the shuffled Id column. You can use `DataFrame.sample()` method to shuffle.

In [ ]:
# code here

### `Q-11:` Find out the maximum and minimum sepal width and petal width of Setosa and Versicolor. To do this:
- First create a dataset with merging the required datasets
- After that, use `groupby` to create groups based on the "Species" column.
- Then find out which are asked in this question.


The output should be like this:
```bash
Minimum Sepal width of Setosa is 2.3
Maximum Sepal width of Setosa is 4.4

**************************************************

Minimum Sepal width of Versicolor is 2.0
Maximum Sepal width of Versicolor is 3.4

**************************************************
```

In [ ]:
# code here